In [100]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import scipy.stats as t
from statsmodels.stats.multicomp import (pairwise_tukeyhsd,
                                         MultiComparison)
from statsmodels.formula.api import ols
import statsmodels.api as sm
from statsmodels.stats.anova import anova_lm

%matplotlib inline

In [101]:
df = pd.read_csv('cars.csv')

### Task 1

Сколько пропущенных значений встретилось в датасете?

In [102]:
df.isna().sum()

car_ID              0
symboling           0
CarName             0
fueltype            0
aspiration          0
doornumber          0
carbody             0
drivewheel          0
enginelocation      0
wheelbase           0
carlength           0
carwidth            0
carheight           0
curbweight          0
enginetype          0
cylindernumber      0
enginesize          0
fuelsystem          0
boreratio           0
stroke              0
compressionratio    0
horsepower          0
peakrpm             0
citympg             0
highwaympg          0
price               0
dtype: int64

### Task 2

Сгенерируем первый признак.

Использовать полное название машины – не самый хороший вариант, поэтому создадим новый признак – марку автомобиля (company). Для этого используйте столбец CarName, разбейте значения ячеек по пробелу и запишите в колонку первый элемент. Например:

'audi 100 ls' → 'audi'
Сколько всего уникальных марок машины встречается в датасете? Столбец CarName с полным названием машины удалите из датасета, а также car_ID, они не пригодятся для дальнейшего анализа.

In [103]:
df['CarName'] = df['CarName'].apply(lambda x: x.split(' ')[0])

In [104]:
print('Answer:', df.CarName.nunique())

Answer: 28


### Task 3

Теперь внимательнее посмотрите на уникальные значения company. Часть из них оказалась с ошибками:

- 'maxda' → 'mazda'
- 'Nissan' → 'nissan'
- 'porcshce' → 'porsche'
- 'toyouta' → 'toyota'
- 'vokswagen' & 'vw' → 'volkswagen'

Сколько уникальных производителей осталось в итоге?

In [105]:
df['CarName'].replace('maxda', 'mazda', inplace=True)

In [106]:
df['CarName'].replace('vw', 'volkswagen', inplace=True)
df['CarName'].replace('vokswagen', 'volkswagen', inplace=True)

In [107]:
df['CarName'].replace('porcshce', 'porsche', inplace=True)

In [108]:
df['CarName'].replace('toyouta', 'toyota', inplace=True)

In [109]:
df['CarName'] = df['CarName'].apply(lambda x: x.lower())

In [110]:
df.rename(columns={'CarName' : 'company'}, inplace=True)

In [111]:
df.drop('car_ID', inplace=True, axis=1)

In [112]:
print('Answer:', df.company.nunique())

Answer: 22


### Task 4

Отлично! Чтобы не перегружать модель большим количеством предикторов, оставим только часть из них:

{'company', 'fueltype', 'aspiration','carbody', 'drivewheel', 'wheelbase', 'carlength','carwidth', 'curbweight', 'enginetype', 'cylindernumber', 'enginesize', 'boreratio','horsepower'}

также не забыв про то, что мы предсказываем – 'price'. 

После этого посчитайте корреляцию между price и другими переменными. Чему равна корреляция между price и horsepower? Ответ округлите до 2 знаков после точки.


In [113]:
df_new = df[['company',
             'fueltype',
             'aspiration',
             'carbody',
             'drivewheel',
             'wheelbase',
             'carlength',
             'carwidth',
             'curbweight',
             'enginetype',
             'cylindernumber',
             'enginesize',
             'boreratio',
             'horsepower',
             'price'
            ]]

In [114]:
df_new.corr(method='pearson')

,wheelbase,carlength,carwidth,curbweight,enginesize,boreratio,horsepower,price
wheelbase,1.000000,0.874587,0.795144,0.776386,0.569329,0.488750,0.353294,0.577816
carlength,0.874587,1.000000,0.841118,0.877728,0.683360,0.606454,0.552623,0.682920
carwidth,0.795144,0.841118,1.000000,0.867032,0.735433,0.559150,0.640732,0.759325
curbweight,0.776386,0.877728,0.867032,1.000000,0.850594,0.648480,0.750739,0.835305
enginesize,0.569329,0.683360,0.735433,0.850594,1.000000,0.583774,0.809769,0.874145
boreratio,0.488750,0.606454,0.559150,0.648480,0.583774,1.000000,0.573677,0.553173
horsepower,0.353294,0.552623,0.640732,0.750739,0.809769,0.573677,1.000000,0.808139
price,0.577816,0.682920,0.759325,0.835305,0.874145,0.553173,0.808139,1.000000


In [115]:
print('Answer:', 0.81)

Answer: 0.81


### Task 5

Последний шаг в подготовке данных: линейная регрессия в python не справляется с категориальными переменными (типом object в pandas), поэтому давайте применим pd.get_dummies(). Пример использования:

df_dummy = pd.get_dummies(data=cars[[список_столбцов_типа_object]], drop_first = True)

Не забудьте присоединить к полученному датасету столбцы с переменными других типов

Сколько колонок теперь имеется в датасете?

In [116]:
df_dummy = pd.get_dummies(data=df_new[['company',
                                       'fueltype',
                                       'aspiration',
                                       'carbody',
                                       'drivewheel',
                                       'enginetype',
                                       'cylindernumber'
                                      ]], drop_first=True)

In [117]:
full_data = pd.concat([df_dummy,
                       df_new[['wheelbase',
                            'carlength',
                            'carwidth',
                            'curbweight',
                            'enginesize',
                            'boreratio',
                            'horsepower',
                            'price'
                              ]]],
                      axis=1)

In [118]:
print('Answer:', full_data.shape[1])

Answer: 49


### Task 6

Сначала построим небольшую модель всего с одним предиктором цены (price) – horsepower.

Какой процент изменчивости объясняет полученная модель? (округлите до целого)

In [121]:
Y = full_data['price']
X = sm.add_constant(full_data.iloc[:, -2:-1])
model = sm.OLS(Y, X)
results = model.fit()

In [122]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.653
Model:                            OLS   Adj. R-squared:                  0.651
Method:                 Least Squares   F-statistic:                     382.2
Date:                Sun, 29 May 2022   Prob (F-statistic):           1.48e-48
Time:                        03:10:06   Log-Likelihood:                -2024.0
No. Observations:                 205   AIC:                             4052.
Df Residuals:                     203   BIC:                             4059.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -3721.7615    929.849     -4.003      0.000   -5555.163   -1888.360
horsepower   163.2631      8.351     19.549      0.000     146.796     179.730
==============================================================================
Omnibus:                       47.741   Durbin-Watson:                   0.792
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               91.702
Skew:                           1.141   Prob(JB):                     1.22e-20
Kurtosis:                       5.352   Cond. No.                         314.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Answer: 65%

### Task 7

Теперь – две модели:

- модель со всеми предикторами
- модель со всеми предикторами, кроме марок машин

Обратите внимание на изменения в $$ R^2 $$, коэффициентах и их значимости. Какую модель лучше оставить? 

In [123]:
Y = full_data['price']
X = sm.add_constant(full_data.iloc[:, :-1])
model = sm.OLS(Y, X)
results = model.fit()

In [124]:
results.summary() # 0.959

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.959
Model:                            OLS   Adj. R-squared:                  0.948
Method:                 Least Squares   F-statistic:                     81.09
Date:                Sun, 29 May 2022   Prob (F-statistic):           4.86e-89
Time:                        03:11:59   Log-Likelihood:                -1804.2
No. Observations:                 205   AIC:                             3702.
Df Residuals:                     158   BIC:                             3858.
Df Model:                          46                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                 -3.472e+04   1.17e+04     -2.959      0.004   -5.79e+04   -1.15e+04
company_audi            437.1040   2125.971      0.206      0.837   -3761.885    4636.093
company_bmw            7993.4998   2105.132      3.797      0.000    3835.671    1.22e+04
company_buick          3431.0969   2472.252      1.388      0.167   -1451.829    8314.023
company_chevrolet     -2107.0167   2054.269     -1.026      0.307   -6164.387    1950.353
company_dodge         -3034.5530   1686.337     -1.799      0.074   -6365.224     296.118
company_honda         -2328.7396   1622.141     -1.436      0.153   -5532.617     875.138
company_isuzu          -926.1643   1819.489     -0.509      0.611   -4519.823    2667.495
company_jaguar         2695.2664   2546.268      1.059      0.291   -2333.847    7724.379
company_mazda         -1291.5657   1652.561     -0.782      0.436   -4555.527    1972.395
company_mercury       -2762.0259   2786.701     -0.991      0.323   -8266.016    2741.965
company_mitsubishi    -3699.6473   1650.046     -2.242      0.026   -6958.640    -440.654
company_nissan        -1488.1217   1563.564     -0.952      0.343   -4576.305    1600.061
company_peugeot       -2671.4595   1795.328     -1.488      0.139   -6217.398     874.479
company_plymouth      -3052.1258   1670.829     -1.827      0.070   -6352.166     247.915
company_porsche        4915.9715   2797.761      1.757      0.081    -609.864    1.04e+04
company_renault       -2865.2197   2073.046     -1.382      0.169   -6959.676    1229.237
company_saab            719.8222   1767.620      0.407      0.684   -2771.390    4211.035
company_subaru        -1.164e+04   3416.221     -3.407      0.001   -1.84e+04   -4892.375
company_toyota        -2608.7871   1539.320     -1.695      0.092   -5649.086     431.512
company_volkswagen    -2261.9355   1606.886     -1.408      0.161   -5435.683     911.812
company_volvo            68.2605   1943.170      0.035      0.972   -3769.679    3906.200
fueltype_gas           -243.7792    908.310     -0.268      0.789   -2037.774    1550.216
aspiration_turbo        596.6566    676.548      0.882      0.379    -739.588    1932.902
carbody_hardtop       -2556.7157   1213.929     -2.106      0.037   -4954.337    -159.094
carbody_hatchback     -3044.4196   1107.064     -2.750      0.007   -5230.972    -857.867
carbody_sedan         -2256.9173   1154.971     -1.954      0.052   -4538.090      24.256
carbody_wagon         -2815.4970   1223.156     -2.302      0.023   -5231.343    -399.651
drivewheel_fwd         -169.4926    888.585     -0.191      0.849   -1924.531    1585.545
drivewheel_rwd          231.9066   1039.685      0.223      0.824   -1821.566    2285.380
enginetype_dohcv      -8358.0756   4326.239     -1.932      0.055   -1.69e+04     186.645
enginetype_l          -

In [133]:
df_dummy = pd.get_dummies(data=df_new[['fueltype',
                                       'aspiration',
                                       'carbody',
                                       'drivewheel',
                                       'enginetype',
                                       'cylindernumber'
                                      ]], drop_first=True)

data_wout_company = pd.concat([df_dummy,
                       df_new[['wheelbase',
                            'carlength',
                            'carwidth',
                            'curbweight',
                            'enginesize',
                            'boreratio',
                            'horsepower',
                            'price'
                              ]]],
                      axis=1)

In [134]:
Y = data_wout_company['price']
X = sm.add_constant(data_wout_company.iloc[:, :-1])
model = sm.OLS(Y, X)
results = model.fit()

In [136]:
results.summary() # 0.914

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.914
Model:                            OLS   Adj. R-squared:                  0.901
Method:                 Least Squares   F-statistic:                     72.32
Date:                Sun, 29 May 2022   Prob (F-statistic):           9.86e-81
Time:                        03:15:09   Log-Likelihood:                -1881.6
No. Observations:                 205   AIC:                             3817.
Df Residuals:                     178   BIC:                             3907.
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                   -1.7e+04    1.3e+04     -1.309      0.192   -4.26e+04    8625.219
fueltype_gas          -2423.0935    975.579     -2.484      0.014   -4348.283    -497.904
aspiration_turbo      -1139.2334    787.230     -1.447      0.150   -2692.738     414.271
carbody_hardtop       -3728.0949   1519.366     -2.454      0.015   -6726.383    -729.807
carbody_hatchback     -4639.1279   1294.594     -3.583      0.000   -7193.855   -2084.401
carbody_sedan         -3337.7854   1356.075     -2.461      0.015   -6013.838    -661.732
carbody_wagon         -4315.4661   1465.089     -2.946      0.004   -7206.646   -1424.287
drivewheel_fwd          297.2402   1121.766      0.265      0.791   -1916.431    2510.912
drivewheel_rwd         1939.1594   1226.536      1.581      0.116    -481.263    4359.582
enginetype_dohcv      -1.713e+04   4540.120     -3.773      0.000   -2.61e+04   -8171.142
enginetype_l           2176.0521   1332.818      1.633      0.104    -454.106    4806.211
enginetype_ohc         4006.5687    897.437      4.464      0.000    2235.584    5777.553
enginetype_ohcf        5421.2822   1388.078      3.906      0.000    2682.076    8160.488
enginetype_ohcv       -5001.2242   1345.435     -3.717      0.000   -7656.280   -2346.169
enginetype_rotor      -5196.6413   2084.532     -2.493      0.014   -9310.217   -1083.066
cylindernumber_five   -1.389e+04   2891.599     -4.802      0.000   -1.96e+04   -8180.353
cylindernumber_four   -1.618e+04   3243.610     -4.989      0.000   -2.26e+04   -9780.800
cylindernumber_six    -1.175e+04   2287.516     -5.135      0.000   -1.63e+04   -7231.547
cylindernumber_three  -9264.0553   4750.899     -1.950      0.053   -1.86e+04     111.279
cylindernumber_twelve -1.214e+04   3386.507     -3.585      0.000   -1.88e+04   -5458.378
cylindernumber_two    -5196.6413   2084.532     -2.493      0.014   -9310.217   -1083.066
wheelbase                71.1868     87.028      0.818      0.414    -100.552     242.925
carlength               -51.3497     50.341     -1.020      0.309    -150.692      47.993
carwidth                541.8700    253.327      2.139      0.034      41.958    1041.782
curbweight                2.9577      1.796      1.647      0.101      -0.585       6.501
enginesize               36.0515     22.376      1.611      0.109      -8.105      80.208
boreratio             -2230.4519   1731.681     -1.288      0.199   -5647.719    1186.815
horsepower               86.8164     16.717      5.193      0.000      53.827     119.806
==============================================================================
Omnibus:                       18.493   Durbin-Watson:                   1.249
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               50.728
Skew:                           0.293   Prob(JB):       

### Task 8

Итого:

Выбранная модель (последняя) объясняет примерно 90 % дисперсии (окр. до целого). Среди предикторов 10 из 27 оказались не значимыми (p > 0.05). Пример интерпретации: при единичном изменении показателя horsepower, цена ВОЗРАСТАЕТ на 86.8164